## FEM for the clover 

In [4]:
from ngsolve import *
from netgen.read_gmsh import ReadGmsh
from netgen.csg import *
import numpy as np
import meshio

In [16]:
#create mesh
mesh = Mesh(ReadGmsh("clover_mesh.msh"))

In [17]:
List_of_mats = ['metal', 'vacuum']

In [18]:
def FEM(epsm,order):
    epsv=1
    eps_val={'metal' : epsm, 'vacuum' : epsv}
    eps=CoefficientFunction([eps_val[mat] for mat in List_of_mats])
    epsinv_val = {'metal' : 1/epsm, 'vacuum' : 1/epsv}
    epsinv=CoefficientFunction([epsinv_val[mat] for mat in List_of_mats])
    fes = H1(mesh, order=order, complex=True)
    

In [20]:
FEM(-1.1,2)